In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df= pd.read_csv('data/income_data_2017_clean_zeros.csv.bz2', compression='bz2', index_col='peridnum')

List of features to consider

In [1]:
features_list = ['peioocc','ern_val', 'a_hga', 'H_NUMPER', 'MARSUPWT', 'FKIND', 'FPERSONS', 
                 'FOWNU6', 'FOWNU18', 'a_age']
features_list = [str.lower(v) for v in features_list]

1. Filtering for individuals eraning between \$1,000 and \$200,000 per year, 
2. Getting the features from the DataFrame, and droping nan's

In [5]:
df2 = df[(df['ern_val'] > 10000) & (df['ern_val'] < 200000) &(df['a_hrs1'] >= 40)]
features_df = df2.loc[:,features_list]
features_df = features_df.dropna()

In [6]:
# Examining the dataset
features_df.describe()

,peioocc,ern_val,a_hga,h_numper,marsupwt,fkind,fpersons,fownu6,fownu18,a_age
count,58439.000000,58439.000000,58439.000000,58439.000000,58439.000000,58439.000000,58439.000000,58439.000000,58439.000000,58439.000000
mean,4233.423228,45706.656616,40.575249,3.237547,149642.294906,1.488732,2.970876,0.254282,0.876196,41.845035
std,2730.958227,29552.755455,2.608772,1.513848,94971.419929,0.762995,1.509773,0.576075,1.102357,11.321998
min,10.000000,10002.000000,31.000000,1.000000,9281.000000,1.000000,1.000000,0.000000,0.000000,18.000000
25%,2010.000000,25000.000000,39.000000,2.000000,74047.000000,1.000000,2.000000,0.000000,0.000000,33.000000
50%,4600.000000,38000.000000,40.000000,3.000000,143627.000000,1.000000,3.000000,0.000000,0.000000,42.000000
75%,6200.000000,57057.500000,43.000000,4.000000,191577.000000,2.000000,4.000000,0.000000,2.000000,50.000000
max,9750.000000,199000.000000,46.000000,15.000000,978808.000000,3.000000,15.000000,5.000000,9.000000,74.000000


In [7]:
features_list.remove('ern_val')
X = features_df[features_list]
Y = ((features_df['ern_val'] > 55000).values)

Splitting Train/Test

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=0)

Extracting sample weights from the data 

In [11]:
weights = X_train['marsupwt']
X_train = X_train.drop('marsupwt',axis=1)
X_test = X_test.drop('marsupwt', axis=1)

Trying a desicsion tree classifier

In [23]:
from sklearn.tree import DecisionTreeClassifier
_=model = DecisionTreeClassifier(max_depth=8)
_=model.fit(X_train, y_train, sample_weight=weights)
print(model.score(X_test, y_test))

0.796064339493


While the score is not terribly bad, the confusion matrix whows a different picture

In [24]:
import sklearn.metrics as met
Y_pred = model.predict(X_test)
met.confusion_matrix(y_test,Y_pred, labels=[0,1])

array([[20002,  1665],
       [ 4294,  3259]])

Trying to adaptive boosting classifier

In [28]:
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 50
_model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
_=model.fit(X_train, y_train, sample_weight=weights)
print(model.score(X_test, y_test))

0.798323066393


In [29]:
Y_pred = model.predict(X_test)
met.confusion_matrix(y_test,Y_pred, labels=[0,1])

array([[20016,  1651],
       [ 4242,  3311]])

A bit better, but still problematic,
Trying a begging classifier

In [31]:
from sklearn.ensemble import BaggingClassifier
_model = BaggingClassifier(n_estimators=num_trees, random_state=seed)
_=model.fit(X_train, y_train, sample_weight=weights)
print(model.score(X_test, y_test))

0.798323066393


In [32]:
Y_pred = model.predict(X_test)
met.confusion_matrix(y_test,Y_pred, labels=[0,1])

array([[20016,  1651],
       [ 4242,  3311]])

Finally, trying a simple naive Bayes  and logistic regression classifiers

In [35]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train,sample_weight=weights)
model.score(X_test, y_test)

0.76054072553045859

In [36]:
Y_pred = model.predict(X_test)
met.confusion_matrix(y_test,Y_pred, labels=[0,1])

array([[18379,  3288],
       [ 3709,  3844]])

In [41]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
_=model.fit(X_train, y_train, sample_weight=weights)
print(model.score(X_test, y_test))

0.756570841889


Suprisingly SVM is the worst of all

In [42]:
from sklearn.svm import SVC
model = SVC()
_=model.fit(X_train, y_train, sample_weight=weights)
print(model.score(X_test, y_test))

0.737885010267
